In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
dataset = pd.read_csv('heart.csv')
dataset

FileNotFoundError: ignored

In [0]:
X = dataset.iloc[:, 0:13].values
Y = dataset.iloc[:, 13].values
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1=LabelEncoder()
X[:,1]=labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_2=LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

NameError: ignored

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense
classifier = Sequential()
classifier.add(Dense(units = 13, kernel_initializer = 'random_uniform', activation = 'relu', input_dim = 13))
classifier.add(Dense(units = 13, kernel_initializer = 'random_uniform', activation = 'relu'))
classifier.add(Dense(units = 11, kernel_initializer = 'random_uniform', activation = 'relu'))
classifier.add(Dense(units = 9, kernel_initializer = 'random_uniform', activation = 'relu'))
classifier.add(Dense(units = 5, kernel_initializer = 'random_uniform', activation = 'relu'))
classifier.add(Dense(units = 3, kernel_initializer = 'random_uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'random_uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics =['accuracy'])
classifier.fit(X_train, Y_train, batch_size = 25, epochs = 300, verbose=1, validation_data=(X_test,Y_test))
scores = classifier.evaluate(X_test,Y_test,verbose=0)
print("%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))

In [0]:
from sklearn.model_selection import StratifiedKFold
nsplits=10
seed = 7
np.random.seed(seed)
cvscores= [] 
kfold = StratifiedKFold(n_splits=nsplits, shuffle=True, random_state=seed)
for train, test in kfold.split(X, Y):
    print "----------------------------------------------"
    sc = StandardScaler()
    X_train=X[train]
    Y_train=Y[train]
    X_test=X[test]
    Y_test=Y[test]
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    history=classifier.fit(X_train, Y_train, batch_size = 25, epochs = 300, verbose=1, validation_data=(X_test,Y_test))
    scores = classifier.evaluate(X_test,Y_test,verbose=0)
    print("%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

In [0]:
print cvscores
print("%.2f%% (+/- %.2f%%)\n" % (np.mean(cvscores), np.std(cvscores)))

In [0]:
nsplits=10
seed = 7
maxepochs=1000
np.random.seed(seed)
cvscores= []
acclist = np.zeros(maxepochs)
vacclist = np.zeros(maxepochs)
kfold = StratifiedKFold(n_splits=nsplits, shuffle=True, random_state=seed)
for train, test in kfold.split(X, Y):
    print "----------------------------------------------"
    sc = StandardScaler()
    X_train=X[train]
    Y_train=Y[train]
    X_test=X[test]
    Y_test=Y[test]
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    history=classifier.fit(X_train, Y_train, batch_size = 25, epochs = maxepochs, verbose=1, validation_data=(X_test,Y_test))
    scores = classifier.evaluate(X_test,Y_test,verbose=0)
    print("%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    acclist=np.add(acclist,history.history['acc'])
    vacclist=np.add(vacclist,history.history['val_acc'])

In [0]:
print cvscores
print("%.2f%% (+/- %.2f%%)\n" % (np.mean(cvscores), np.std(cvscores)))

In [0]:
plt.plot(acclist/nsplits)
plt.plot(vacclist/nsplits)
plt.title('Average model accuracy for 500 epochs')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
plt.plot(acclist/vacclist)
plt.title('Training/Testing')
plt.ylabel('ratio')
plt.xlabel('epoch')
plt.show()